<h1>Introduction to Linear Regression (cont'd)</h1>

<h2>Key Concepts</h2>

- Goodness of fit
- Variable transformations
- Multivariate regression
- Categorical variables
- Regularization

<h2>Checking the Goodness of Fit</h2>

In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

Let's load the real estate dataset we were looking at last week:

In [ ]:
data = pd.read_csv("../datasets/real_estate.csv")
data.head()

<div style="border: 3px solid green; padding: 10px">
<b>Exercise 1:</b> Use the statsmodel API to fit a simple linear regression model of price as a function of size.
</div>

In [ ]:
data["Constant"] = 1
m = sm.OLS(..., ...).fit()
m.summary()

The linear model makes two critical assumptions (there are others, but these are two of the most important):

- <b>Homoskedasticity</b>: This is a fancy way of saying that the variance of the errors is the same no matter what the value of the predictor variable is.  In other words, the predicted range of house prices is about as wide for a 1,000 square foot house as it is for a 10,000 square foot house.
- <b>Independence of errors</b>: This means that knowing how much the model gets one house price wrong shouldn't tell us anything about how much the model gets wrong the prices of other houses.

Both of these assumptions are quite strong and rarely hold in practice: the goal is more to make sure they hold reasonably well.  Let's look at the residuals to see how well they hold:

In [ ]:
%matplotlib inline
data["Residual"] = m.resid

fig = plt.figure(figsize=(10, 6))
plt.plot(data["Size"], data["Residual"], "o", markersize=4)
plt.xlabel("Size (sq. ft.)")
plt.ylabel("Actual minus Predicted Price ($)")
plt.title("Size versus Residual Price")
plt.grid()

It doesn't take college-level statistics to tell that neither of the assumptions above hold that well.  First, the errors are clearly much larger for larger houses than for smaller ones.  Second, it looks like we consistently underpredict prices for houses smaller than 1,000 square feet and consistently overpredict prices for houses between 2,000 and 4,000 square feet.

To fix this problem, let's look at histograms of the predictors and responses:

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
data["Size"].plot(kind="hist", rot = -15, bins = 30)
plt.title("Size (sq. ft.)")
plt.grid()
plt.subplot(1, 2, 2)
data["Price"].plot(kind="hist", rot = -15, bins = 30)
plt.grid()
plt.title("Price ($)")

Both of these variables have heavy right tails.  This is a not-infrequent occurrence with variables like size and price that can only be positive.  With a variable like price, it's probably more reasonable to expect that errors will be in percentage rather than absolute bands.  While we may be able to predict the error on a \$500,000 house to within $\pm$\$30,000, it would be unrealistic to expect that for King Leopold's house.  However, perhaps we can predict both house prices to within 6%.

This suggests transforming both prices and sizes logarithmically:

In [ ]:
data["logPrice"] = np.log(data["Price"])
data["logSize"] = np.log(data["Size"])

We plot the histograms of the log transformed data. The distribution looks more normal now.

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
data["logSize"].plot(kind="hist", rot = -15, bins = 30)
plt.title("Log Size (sq. ft.)")
plt.grid()
plt.subplot(1, 2, 2)
data["logPrice"].plot(kind="hist", rot = -15, bins = 30)
plt.grid()
plt.title("Log Price ($)")

Now we can fit the model:

$$ \log \mathrm{Price} = a \cdot \log \mathrm{Size} + b $$:

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 2:</b> Fit a linear model to the log transformed data. How much does the R-squared change?
</div>

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 3:</b> Plot the residuals of the log transformed model.
</div>

To actually get the house price prediction, we can exponentiate both sides of the fitted model:

$$ \mathrm{price} = e^{4.8892} \cdot \mathrm{size}^{1.0491} \approx = 132.8 \cdot \mathrm{size}^{1.0491} $$

So prices are pretty close to linear in house size but grow just a bit faster.

<h2>Multivariate Regression</h2>

Let's turn back to the movie dataset.

In [ ]:
movie_data = pd.read_csv("../datasets/tmdb_data.csv")
movie_data.head()

We want to see if we can predict the vote average for movies using a multivariate linear model:

$$ y_i = \sum_{j=1}^k a_j x_i + b + \epsilon_i $$

by minimizing the loss:

$$ \mathrm{loss} = \sum_{i=1}^N \left(y_i - \sum_{j=1}^k a_j x_i - b\right)^2 $$



Let's clean the data, by removing NA rows and 0 budget movies. Next, let's shuffle the data and take only the first half of the data to build our model. This is called the "in-sample". We'll save the second half of the data, called the "out-of-sample", for later.

In [ ]:
movie_data = movie_data.sample(frac=1, replace = False)
movie_data = movie_data.dropna()
movie_data = movie_data[movie_data.budget != 0]

in_sample = movie_data[0:round(movie_data.shape[0]/2)]
out_sample = movie_data[round(movie_data.shape[0]/2):]
in_sample['constant'] = 1
out_sample['constant'] = 1

in_sample[["vote_average","duration", "budget", "title_year", "constant"]].head()

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 4:</b> Using the in-sample data, fit a multivariate model of the vote average as a function of duration, budget, and title year, including an intercept (constant).
</div>

It seems people prefer longer, lower budget, and earlier movies. Let's look at the residuals for this model against the predicted values:

In [ ]:
fig = plt.figure(figsize=(10, 6))
plt.plot(m.predict(in_sample[["duration", "budget", "title_year", "constant"]]), in_sample["Residual"], "o", markersize=4)
plt.xlabel("Predicted Vote Average")
plt.ylabel("Residual Vote Average")
plt.title("Predicted Vote Average vs Residual Vote Average")
plt.grid()

### Categorical variables 

We can add "categorical" variables to our model. The example below illustrates how we can add the country as a variable.

In [ ]:
for country in in_sample.country.unique():
    print(country)
    in_sample[country] = 0.0
    out_sample[country] = 0.0

    in_sample.loc[in_sample.country == country, country] = 1.0
    out_sample.loc[out_sample.country == country, country] = 1.0
 
#An alternative approach
#in_sample = in_sample.merge(pd.get_dummies(in_sample.country), how='outer', left_index=True, right_index=True)
#out_sample = out_sample.merge(pd.get_dummies(out_sample.country), how='outer', left_index=True, right_index=True)

In order for our design matrix to have full rank, we need to drop one of the countries. Let's choose the US. This will now serve as our baseline against which comparisons will be made, called our "reference level".

In [ ]:
in_sample.drop(labels='United States of America', axis=1, inplace=True)

Now, we're ready to fit our model:

In [ ]:
predictors = ["duration", "budget", "title_year", "constant"] + [c for c in in_sample.country.unique() if c != 'United States of America']
m = sm.OLS(in_sample["vote_average"], in_sample[predictors]).fit()
m.summary()

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 5:</b> Add a categorical variable representing whether the movie is an action movie or not and refit the model.
</div>

In [ ]:
# Hint: use in_sample['genres'].str.contains('Action')

### Regularization

As we add new variables, we improve the R-squared of the model, as our model fits our data better. However, by doing so, we risk "overfitting" to our data. 

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 6:</b> Take a minute to think about why over-fitting is bad.
</div>

While there are many ways to handle overfitting -- many more than can be covered here -- one way is to try to force the slopes in the linear model to be zero unless the predictor really matters.  One way to do this, popularized in the mid-1990s, is to add a term to the sum of squares loss function that penalizes large slopes:

$$ \mathrm{loss} = \sum_{i=1}^N \left(y_i - \sum_{j=1}^k a_j x_i - b\right)^2 + \lambda \sum_{j=1}^k |a_j| $$

where $\lambda$ is called a <b>regularization coefficient</b>.  This is called a <b>lasso regression</b>.  Unfortunately, the fact that we penalize the absolute values (rather than, say, the squares) of the slopes means we can no longer write down closed-form solutions for the optimal slopes and intercept.  However, fast algorithms exist to solve them.

We don't know what value for $\lambda$ we should choose. A popular way is to look at the out-of-sample data and choose the lambda which minimizes the error in the out-of-sample.

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 7:</b> Compute the mean squared error of the model on the out-of-sample.
</div>

In [ ]:
# Hint: use out_sample['vote_average_hat'] = m.predict(...)

<div style="border: 3px solid green; padding: 10px">
  <b>Exercise 8:</b> Loop over some valus for $\lambda$ and plot the out-of-sample mean squared error.
</div>

In [ ]:
mses = []
for regularization in range(30):
    m = sm.OLS(..., ...).fit_regularized(alpha = regularization)
    ...
    ...
    
fig = plt.figure(figsize=(10, 6))
plt.bar(range(len(mses)), mses)
plt.xlabel('Regularization')
plt.ylabel('Mean squared error')
plt.grid()

From the chart above we see that regularization can help reducing the out-of-sample error. In a way, the regularization "simplifies" our model, and for predictions, a simpler model can often perform better than a more complex one, which tends to "overfit" to the in-sample data. 